In [32]:
#'random_500/random_result'
MARKDOWN_DIR = '/home/avybornov/git/twnews/dataset_markdown/'

files = ['1_filtred_alpha/out',
         '2_filtred_beta/out',
         '3_filtred/0/out',
         '3_filtred/1000/out',
         '3_filtred/2000/out',
         '3_filtred/3000/out',
         '3_filtred/4000/out',
]
tweet_texts = {}
news_texts = {}

def parse_tweet(rows, idx):
    if not rows[idx][0].isdigit(): raise(Exception('Incorrect first row of tweet: %s' % idx))
    text = rows[idx]
    
    idx += 1
    #print idx
    
    while not rows[idx][0].isdigit():
        text += ' ' + rows[idx]
        idx += 1
    
    if not rows[idx][0].isdigit(): raise(Exception('Incorrect second row of tweet: %s' % idx))    
    tweet_id = int(rows[idx])
    idx += 1
        
    while rows[idx][0] != '\t':
        if rows[idx][:4] != 'http': raise(Exception('Incorrect row with link of tweet: %s' % idx))
        idx += 1
        
    tweet_texts[tweet_id] = text
    return idx, tweet_id


def parse_news(rows, idx):
    marked = False
    
    if rows[idx][0] != '\t': raise(Exception('News row started without tabulation: %s' % idx))    
    if rows[idx][1] == "!": marked = True
    text = rows[idx]
    idx += 1    
    
    if rows[idx][0] != '\t': raise(Exception('News row started without tabulation: %s' % idx))    
    if rows[idx][1:5] != 'http': raise(Exception('Incorrect row with link of news: %s' % idx))    
    link = rows[idx][1:]
    idx += 1
    
    #print rows[idx]
    if rows[idx][1] != "-": raise(Exception('Incorrect row with news separator: %s' % idx))        
    idx += 1
    
    if marked: news_texts[link] = text
    return idx, link if marked else None


def parse_all_news(rows, idx):
    news_link = None
    
    #print rows[idx] == '\t'
    while idx < len(rows) and rows[idx] and rows[idx][0] == '\t':
        idx, link = parse_news(rows, idx)
        if link and not news_link:
            news_link = link        
        
    return idx, news_link

tweet_to_news = {}
for file_suffix in files:
    filepath = MARKDOWN_DIR + file_suffix
    print filepath
    rows = open(filepath).read().splitlines()
    
    tweet_to_news[file_suffix] = {}
    idx = 0
    while idx < len(rows):
        idx, tweet_id = parse_tweet(rows, idx)
        idx, news_link = parse_all_news(rows, idx)
        tweet_to_news[file_suffix][tweet_id] = news_link
        
        #print idx
#         if idx > 40:
#             break

/home/avybornov/git/twnews/dataset_markdown/1_filtred_alpha/out
/home/avybornov/git/twnews/dataset_markdown/2_filtred_beta/out
/home/avybornov/git/twnews/dataset_markdown/3_filtred/0/out
/home/avybornov/git/twnews/dataset_markdown/3_filtred/1000/out
/home/avybornov/git/twnews/dataset_markdown/3_filtred/2000/out
/home/avybornov/git/twnews/dataset_markdown/3_filtred/3000/out
/home/avybornov/git/twnews/dataset_markdown/3_filtred/4000/out


In [33]:
already_marked = set([])

linked_for_files = {}

for file_suffix, data in tweet_to_news.iteritems():
    print file_suffix, len(data)
    linked = {}
    for tweet_id, link in data.items():
        already_marked.add(tweet_id)
        if link: linked[tweet_id] = link
    linked_for_files[file_suffix] = linked
    print 'linked: %s %.2f%%' % (len(linked), len(linked)*100.0/len(data))
        #print '\t', 'https://twitter.com/statuses/%s' % tweet_id, link
        
from twnews.utils.memoize import dump
dump(already_marked, 'already_marked')


2_filtred_beta/out 1173
linked: 254 21.65%
3_filtred/2000/out 1000
linked: 168 16.80%
3_filtred/3000/out 1000
linked: 171 17.10%
3_filtred/1000/out 1000
linked: 150 15.00%
3_filtred/4000/out 1000
linked: 146 14.60%
3_filtred/0/out 1000
linked: 172 17.20%
1_filtred_alpha/out 1200
linked: 555 46.25%


In [35]:
for file_suffix, data in linked_for_files.items()[-3:-2]:
    print file_suffix
    for tweet_id, link in data.items()[:20]:
        print '\t',tweet_texts[tweet_id]
        print '\t',news_texts[link]

#print len(already_marked)

3_filtred/4000/out
	655) #kp_ru Замгенпрокурора прилетел с проверкой на рыбокомбинат на острове Шикотан  #news
		!Замгенпрокурора проверил цеха рыбокомбината на острове Шикотан
	524) Участники референдума в Нидерландах не знают, где находится Украина – СМИ
		!Участники референдума в Нидерландах не смогли найти Украины на карте
	106) 16 апреля  пройдет Тотальный диктант
		!Акция по проверке знаний русского языка "Тотальный диктант"
	501) «Боруссия» и «Ливерпуль» разошлись миром в матче четвертьфинала ЛЕ: Соперничество дортмундской «Боруссии» и ан... 
		!"Боруссия" и "Ливерпуль" разошлись миром в матче четвертьфинала ЛЕ
	694) Введение в оборот в России новых банкнот достоинством 200 и 2000 рублей объясняется инфляцией в стране. Об этом заявил бывший председатель Ц
		!ЦБ РФ выпустит банкноты достоинством 2000 и 200 рублей
	702) Азербайджан обвинил армянских военных в нарушении перемирия 
		!Минобороны Азербайджана обвинило Армению в нарушении перемирия
	754) Голосование на парламентских в

In [36]:
import random
import shelve

from twnews.utils.memoize import load
from twnews.dataset.storage import TweetsStorage, NewsStorage
from twnews.dataset.url_resolver import UrlResolver
from twnews.dataset.texts import Tweet
from twnews import defaults
from twnews.utils.text_processors import Lemmatizer
from twnews.resolver import get_domain
corpus, _  = load('tf_idf_corpus')
corpus = set(corpus)
lemmatizer = Lemmatizer()
data_shelve = shelve.open(defaults.TWEETS_PATH)
resolver = UrlResolver()

2016-05-11 21:18:57.432558: Start of loading url_map from /home/avybornov/twnews_data_april/resolve_url_map.shelve
2016-05-11 21:18:57.432914: Url map successfully loaded


In [37]:
import os
for file_suffix, data in linked_for_files.items():
    filepath = MARKDOWN_DIR+file_suffix
    dirname, filename = os.path.dirname(filepath), os.path.basename(filepath)

    tweets = []
    for tweet_id, link in data.items():
        tweet = Tweet(data_shelve[str(tweet_id)])
        tweet.resolve_urls(resolver)
        tweet.urls.append(link)
        tweets.append(tweet)
        
    dump(tweets, 'manual_tweets', dirname)
    